# 05tools_2: Understanding Model Performance and Fairness with What-If Tool (WIT)

The [What-If Tool (WIT)](https://pair-code.github.io/what-if-tool/) helps understand model performance across a wide range of inputs.  In this notebook the 05 series models will be evaluated with the WIT tool.

This notebook will show how to connect the tool to the model multiple ways and how to use the tools to evaluate the model.

Model Serving:
- [ ] Vertex AI Endpoint
- [ ] Vertex AI Model
- [ ] TensorFlow ModelServer Locally hosted
- Compare Models
- Incorporate Explainations

---
## Setup

inputs:

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [2]:
REGION = 'us-central1'
DATANAME = 'fraud'
NOTEBOOK = '05tools_2'
SERIES = '05'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

packages:

In [3]:
from google.cloud import aiplatform
from google.cloud import bigquery

from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

import witwidget
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

import numpy as np

clients:

In [4]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client()

parameters:

In [5]:
BUCKET = PROJECT_ID
DIR = f"temp/{NOTEBOOK}"

environment:

In [6]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Get Vertex AI Endpoint And Deployed Model

In [7]:
endpoints = aiplatform.Endpoint.list(filter = f"display_name={SERIES}_{DATANAME}")
endpoint = endpoints[0]

In [8]:
endpoint.display_name

'05_fraud'

In [9]:
model = aiplatform.Model(
    model_name = endpoint.list_models()[0].model+f'@{endpoint.list_models()[0].model_version_id}'
)

In [10]:
model.display_name

'05i_fraud'

In [11]:
model.versioned_resource_name

'projects/1026793852137/locations/us-central1/models/model_05i_fraud@1'

In [12]:
model.uri

'gs://statmike-mlops-349915/fraud/models/05i/20220728003419/18/model'

## Get Data for Model Exploration
Retrive the test data for this series:

In [25]:
pred = bq.query(query = f"SELECT * EXCEPT({VAR_TARGET}), {VAR_TARGET} FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' ORDER BY {VAR_TARGET} DESC").to_dataframe()
pred = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+['splits'])]]
pred.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,148074,-2.219219,0.727831,-5.458230,5.924850,3.932464,-3.085984,-1.677870,0.865075,-3.177260,...,0.417472,-0.817343,-0.028752,0.025723,-0.825835,-0.013089,0.413291,-0.131387,0.0,1
1,129668,0.753356,2.284988,-5.164492,3.831112,-0.073622,-1.316596,-1.855495,0.831079,-1.567514,...,0.382007,0.033958,0.187697,0.358433,-0.488934,-0.258802,0.296145,-0.047174,2.0,1
2,56887,-0.075483,1.812355,-2.566981,4.127549,-1.628532,-0.805895,-3.390135,1.019353,-2.451251,...,0.794372,0.270471,-0.143624,0.013566,0.634203,0.213693,0.773625,0.387434,5.0,1
3,146998,-2.064240,2.629739,-0.748406,0.694992,0.418178,1.392520,-1.697801,-6.333065,1.724184,...,6.215514,-1.276909,0.459861,-1.051685,0.209178,-0.319859,0.015434,-0.050117,8.0,1
4,78725,-4.312479,1.886476,-2.338634,-0.475243,-1.185444,-2.112079,-2.122793,0.272565,0.290273,...,0.550541,-0.067870,-1.114692,0.269069,-0.020572,-0.963489,-0.918888,0.001454,60.0,1


In [87]:
newobs = pred.to_dict(orient='records')
len(newobs)

28522

In [88]:
newobs[0]

{'Time': 32799,
 'V1': 1.15347743766561,
 'V2': -0.0478588055804616,
 'V3': 1.35836288729212,
 'V4': 1.48062009487976,
 'V5': -1.22259808550513,
 'V6': -0.481689608379461,
 'V7': -0.6544612667240159,
 'V8': 0.128114599402494,
 'V9': 0.907094671648477,
 'V10': -0.0364175882073298,
 'V11': -0.659895142180526,
 'V12': -0.307334930019039,
 'V13': -1.3656343720183501,
 'V14': 0.035305634441467997,
 'V15': 0.7114399756440071,
 'V16': 0.25384340384905,
 'V17': -0.22283165732475999,
 'V18': 0.26559908021792394,
 'V19': -0.5970199308963089,
 'V20': -0.24568562944008399,
 'V21': 0.12551444308840598,
 'V22': 0.480049308608633,
 'V23': -0.0259637518455547,
 'V24': 0.7018428515999721,
 'V25': 0.41724451503333504,
 'V26': -0.257690694038964,
 'V27': 0.06011458360133701,
 'V28': 0.0353320583585812,
 'Amount': 0.0,
 'Class': 0}

In [26]:
examples = pred.values.tolist()
examples_names = pred.columns.values.tolist()

In [27]:
examples[0]

[148074.0,
 -2.21921860215056,
 0.7278314111063922,
 -5.45822994652182,
 5.92484984705884,
 3.9324638237634395,
 -3.0859842366267003,
 -1.67786998770016,
 0.865074610405235,
 -3.1772602889458597,
 -3.4192073840566404,
 3.6931739422441203,
 -3.97843975507806,
 -1.71859087457346,
 -8.636297393652589,
 -0.24296482145526502,
 1.17488417316765,
 2.13460635695284,
 2.59436483300614,
 -1.25758897993879,
 0.9647718037347099,
 0.41747174595057,
 -0.8173433840569749,
 -0.0287524020141088,
 0.0257225108657227,
 -0.8258353432218559,
 -0.0130890304987416,
 0.413291188715315,
 -0.131387346404896,
 0.0,
 1.0]

## A Python Function For Predictions
The WIT tool connects to models for prediction is several ways.  To demonstrate custom prediction functions, this section builds a Python function that calls a Vertex AI Endpoint.

In [83]:
def remote_predictor(obs):
    if type(obs) is dict: obs = [obs]
    predictions = []
    for ob in obs:
        ob.pop(VAR_TARGET, None)
        instances = [json_format.ParseDict(ob, Value())]
        predictions.append(endpoint.predict(instances=instances).predictions[0])
    return predictions

In [84]:
remote_predictor(newobs[0:2])

[[0.999275744, 0.000724321057], [0.999849558, 0.000150473832]]

In [28]:
def remote_predictor(obs):
    predictions = []
    for ob in obs:
        example = dict(zip(examples_names, ob))
        example.pop(VAR_TARGET, None)
        instances = [json_format.ParseDict(example, Value())]
        predictions.append(endpoint.predict(instances=instances).predictions[0])
    return predictions

In [29]:
remote_predictor(examples[0:2])

[[0.00166473, 0.998335302], [0.0168009363, 0.98319906]]

The WIT tool expect predictions that match the target. In this case, a binary classification model, the response should be a list of probabilities for the two classes.  This function will adjust the response from the `remote_predictor` function:

---
## Configure WIT
The What-If Tool expects two things: a set of examples to profile the model and a link to the model to create the "what-if .." predictions during the model evaluation.

Using Vertex AI Model:
- Guide for [WitConfigBuilder](https://github.com/pair-code/what-if-tool/blob/master/witwidget/notebook/visualization.py)

In [39]:
examples[100]

[33946.0,
 1.04705069762402,
 0.0552054899555905,
 1.59878821386702,
 2.9447030749121295,
 -0.837250342267749,
 0.660945697667275,
 -0.7114259532997529,
 0.34916361249899996,
 0.5062099357452171,
 0.3537707067129221,
 -0.9278852626153911,
 0.264724874167351,
 -0.68170102800861,
 -0.580548197081264,
 -1.2058638343728703,
 -0.334974539429182,
 0.428017439336503,
 -0.8062391873457428,
 -1.0439429337186301,
 -0.26141112870822897,
 0.0642399701614924,
 0.5573775682908101,
 -0.052924958944252996,
 0.442928103182081,
 0.4848687098523621,
 0.224057670073232,
 0.0625195212074007,
 0.0258242416694256,
 0.0,
 0.0]

In [ ]:
config_builder = (
    WitConfigBuilder(examples[0:100], examples_names)
    .set_custom_predict_fn(remote_predictor)
    .set_target_feature(f'"{VAR_TARGET}"')
    .set_label_vocab(['Normal', 'Fraud'])
)
WitWidget(config_builder, height = 800)

---
## Explore The Model

---
## Screenshot Walkthrough
The interactive tool widget above can be use to reproduce these guided explorations: